# Local Search
## Solving N-queen problem with min-conflict algorithm

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts
    
    * Reassign the value of this variable to another value that with the least constraint onflicts with other variables

In [7]:
import numpy as np
import time


def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def next_action(self, point):
        i, j = point
        if 0 <= i < self.n and 0 <= j < self.n and i * self.n + j < self.n ** 2 - 1:
            j += 1
            if j == self.n:
                j = 0
                i += 1
            return i, j
        else:
            return None

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

In [8]:
import random
import sys

def init_stack(n):
    action_stack = []
    possible_col = np.arange(n)
    for i in range(n):
        a = np.random.choice(possible_col)
        possible_col = np.delete(possible_col, np.where(possible_col == a))
        action_stack.append((i, a))
    return action_stack

def change(action_stack, row, col):
    """
    将第row行的皇后移动到第col列
    """
    action_stack[row] = (row, col)

def is_conflict(point_a, point_b):
    (x_d, y_d) = (abs(point_a[0] - point_b[0]), abs(point_a[1] - point_b[1]))
    if x_d == y_d or x_d == 0 or y_d == 0:
        return True
    return False


def min_conflict(problem):

    def count_conflict_num(point, action_stack):
        """
        计算多少个皇后会与位于point处的皇后冲突
        """
        cnt = 0
        for pos in action_stack:
            if action_stack[point[0]] != pos:
                if is_conflict(point, pos):
                    cnt += 1
        return cnt
    
        
    def choose_to_change(row, action_stack):
        """
        计算应当把row行处的皇后移动到哪里,并进行移动
        """
        conflict_nums = []
        for i in range(len(action_stack)):
            conflict_nums.append(count_conflict_num((row, i), action_stack))
        min_num = min(conflict_nums)
        min_idx = [i for i,x in enumerate(conflict_nums) if x == min_num]
        better_pos = min_idx[random.randint(0, len(min_idx) - 1)]
        return change(action_stack, row, better_pos)
        
        
    action_stack = init_stack(problem.n)
    last_row = 0
    

    while not problem.is_satisfy(action_stack):
        
        conflict_points = []
        
        for point in action_stack:
            for point_2 in action_stack:
                if point != point_2:
                    if is_conflict(point, point_2):
                        conflict_points.append(point)
        
        point = random.choice(conflict_points)
        # print(conflict_points)
            
        temp_row = point[0]
        choose_to_change(temp_row, action_stack)
        last_row = temp_row
            
        yield action_stack
    


In [9]:
# test block
n = 100
render = (n == 7)
p = Problem(n)
if render:
    import pygame

    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = min_conflict(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen_s.png')
    queen_img = pygame.transform.scale(queen_img, (80, 80))
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
else:
    start_time = time.time()
    for actions in min_conflict(p):
        pass
    print(time.time() - start_time)
    p.print_state(actions)

2.093794107437134
_________________________________________________________________________________________________________________________________________________________________________________________________________
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·